# Exercício 2 - N-Gramas

Neste exercício, vamos construir um modelo linguístico baseados em n-gramas. Para criarmos esse modelo, usaremos a frequência relativa para calcular a probabilidade de uma palavra aparecer dado uma sequência de palavras. Com o modelo, vamos gerar um texto baseado no corpus de treinamento.

In [0]:
from nltk.corpus import machado
from nltk.util import bigrams, trigrams, ngrams
from collections import Counter
import pandas as pd
import random
import nltk
nltk.download('machado')
nltk.download('punkt')

[nltk_data] Downloading package machado to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Vamos utilizar um corpus em português para gerar nosso modelo. Vamos carregar, a partir do corpus Machado do NLTK, o texto Memórias Póstumas de Brás Cubas. Vamos carregar uma lista de sentenças tokenizadas.

In [0]:
corpus = machado.sents('romance/marm05.txt')
corpus[:100]

[['Romance', ',', 'Memórias', 'Póstumas', 'de', 'Brás', 'Cubas', ',', '1880'], ['Memórias', 'Póstumas', 'de', 'Brás', 'Cubas'], ...]

Para gerar o modelo corretamente, precisamos demarcar onde a sentença começa e onde termina. Vamos adicionar tokens de início e fim de sentença. Vamos criar um modelo de trigrama, assim, o primeiro trigrama precisa demarcar a primeira palavra da sentença. Por isso, adicionamos dois tokens de início em cada sentença e um de fim no final. Aqui também convertemos a nossa lista de sentenças em uma lista de palaveras.

In [0]:
words = []
for sent in corpus:
    sent.insert(0, '<s>')
    sent.insert(0, '<s>')
    sent.append('</s>')
    words.extend(sent)

Vamos agora gerar as trigramas e bigramas necessárias para o modelo. Para isso, usamos a função bigrams e trigrams do pacote NLTK que facilita nosso trabalho. A função funciona a partir de listas de palavras previamente segmentadas.

In [0]:
words_bigram = bigrams(words)
words_trigram = trigrams(words)

Vamos contar os bigramas, para isso vamos usar o Counter.

In [0]:
count_bigram = Counter(words_bigram)
count_bigram

Counter({('<s>', '<s>'): 3822,
         ('<s>', 'Romance'): 1,
         ('Romance', ','): 1,
         (',', 'Memórias'): 1,
         ('Memórias', 'Póstumas'): 5,
         ('Póstumas', 'de'): 4,
         ('de', 'Brás'): 6,
         ('Brás', 'Cubas'): 29,
         ('Cubas', ','): 16,
         (',', '1880'): 1,
         ('1880', '</s>'): 1,
         ('</s>', '<s>'): 3821,
         ('<s>', 'Memórias'): 1,
         ('Cubas', '</s>'): 1,
         ('<s>', 'Texto'): 1,
         ('Texto', '-'): 1,
         ('-', 'fonte'): 1,
         ('fonte', ':'): 1,
         (':', 'Obra'): 1,
         ('Obra', 'Completa'): 1,
         ('Completa', ','): 1,
         (',', 'Machado'): 1,
         ('Machado', 'de'): 2,
         ('de', 'Assis'): 2,
         ('Assis', ','): 1,
         (',', 'Rio'): 1,
         ('Rio', 'de'): 8,
         ('de', 'Janeiro'): 8,
         ('Janeiro', ':'): 1,
         (':', 'Editora'): 1,
         ('Editora', 'Nova'): 1,
         ('Nova', 'Aguilar'): 1,
         ('Aguilar', ','): 1,


Precisamos também contar os trigramas, assim, use o Counter nos trigramas na linha abaixo.

In [0]:
count_trigram = Counter(words_trigram)

Criaremos uma lista de bigramas e trigramas únicos para podermos gerar nosso modelo. Utilizaremos essa lista como chave do nosso dicionário de contagem.

In [0]:
bigram_key = list(count_bigram.keys())
trigram_key = list(count_trigram.keys())

Para o cálculo da frequência relativa de trigramas, precisamos, para cada trigrama, pegar o bigrama formado pelas primeiras duas palavras do trigrama. Assim, calculamos a probabilidade da última palavra do trigrama surgir no corpus, dado a aparição das duas primeiras palavras no trigrama:

<center>$P(w_{3} | w_{1},w_{2}) = \frac{c(w_{1}, w_{2}, w_{3})}{c(w_{1}, w_{2})}$</center>

Assim, para cada trigrama único, pegamos o bigrama correspondente e calculamos a probabilidade acima. Vamos colocar todos esses dados em um dataframe pandas, por isso, vamos criar listas com os bigramas correspondentes para cada trigrama (primeira e segunda palavra no trigrama) e a última palavra do trigrama, formando uma tabela.

In [0]:
list_bigram = []
list_lastword = []
list_probs = []

for t in trigram_key:
    key_bigram = (t[0], t[1])
    prob = count_trigram[t] / count_bigram[key_bigram]
    list_bigram.append(key_bigram)
    list_lastword.append(t[2])
    list_probs.append(prob)

A partir das listas criadas acima, vamos criar um dataframe pandas. Esse dataframe será nosso modelo linguístico.

In [0]:
model_df = pd.DataFrame({'bigram': list_bigram, 'lastword': list_lastword, 'prob': list_probs})
model_df

,bigram,lastword,prob
0,"(<s>, <s>)",Romance,0.000262
1,"(<s>, Romance)",",",1.000000
2,"(Romance, ,)",Memórias,1.000000
3,"(,, Memórias)",Póstumas,1.000000
4,"(Memórias, Póstumas)",de,0.800000
5,"(Póstumas, de)",Brás,1.000000
6,"(de, Brás)",Cubas,0.833333
7,"(Brás, Cubas)",",",0.482759
8,"(Cubas, ,)",1880,0.062500
9,"(,, 1880)",</s>,1.000000


Vamos testar nosso dataframe. Crie um dataframe contendo apenas o bigrama ('Brás', 'Cubas').

In [0]:
bigram_df = model_df.loc[model_df['bigram'] == ('Brás', 'Cubas')]
bigram_df

,bigram,lastword,prob
7,"(Brás, Cubas)",",",0.482759
14,"(Brás, Cubas)",</s>,0.034483
92,"(Brás, Cubas)",foi,0.034483
179,"(Brás, Cubas)",são,0.034483
211,"(Brás, Cubas)",(,0.034483
320,"(Brás, Cubas)",se,0.034483
338,"(Brás, Cubas)",um,0.034483
716,"(Brás, Cubas)",.,0.206897
27329,"(Brás, Cubas)",;,0.068966
39094,"(Brás, Cubas)",não,0.034483


Agora mostre que temos um modelo probabilístico. Para que um modelo seja probabilístico, precisamos que todas as probablidades de um dado bigrama somem 1. Complete a linha abaixo somando as probabilidades do dataframe criado acima. A soma deve ser aproximadamente 1.

In [0]:
sum(bigram_df['prob'])

0.9999999999999998

Com o nosso modelo criado, vamos usá-lo para gerar texto. Para gerar texto temos que iniciar com o bigrama de início de sentença, pegar todos os trigramas que iniciam com esse bigrama e tomar a próxima palavra de acordo com sua probabilidade. O próximo bigrama será as duas últimas palavras do trigrama escolhido, isto é, pego a última palavra do bigrama atual e junto com a próxima palavra escolhida de acordo com a probabilidade. Complete o código abaixo para gerar o texto.

In [0]:
num_sents = 50
current_bigram = ('<s>', '<s>')
i = 0
while i < num_sents:
    df = model_df.loc[model_df['bigram'] == current_bigram]
    words = df['lastword'].values
    probs = df['prob'].values
    last_word = random.choices(words, probs)[0]
    
    current_bigram = (current_bigram[1], last_word)
    
    if last_word == '</s>':
        i+=1
    
    if last_word != '<s>' and last_word != '</s>':
        print(last_word, end=' ')

Fui ao enterro . CAPÍTULO CXXXVIII / A PROPÓSITO DE BOTAS Minha irmã encaminhou a candidatura de Lobo Neves , e estarás perdida ... morta ... e ele havia de mostrar de que o do Bocage ! Que há entre as unhas , que vence em eficácia o cálculo humano , e todos abençoarão esse canto de uma rígida e meiga companheira do homem ; meu cérebro , porque eu não era outra coisa além dos livros , e que antes , só a custo pela escada da direita , toda fechada , menos luzidios que os esperava um pouco turvados ; mas não sei se andaste bem ou mal ; esse entrou a rolar como a moeda , hem ? Capistrano de Abreu , noticiando a publicação de um modo malicioso ; talvez queiras uma coisa redonda e amarela . Creio que essa : orçam todas pela mesma vulgaridade ou fraqueza . Comíamos , é uma operação conveniente , como Ezequiel ; logo que chegou a me dar o sol , se não pode continuar assim , e , com quem dançara num célebre baile da Praia do Flamengo e de morte , é verdade ; se te não agradar , pago - me a boa

Agora é a sua vez! Vamos criar um modelo mais performático para o corpus. Para isso, vamos usar 5-gramas. Usando a função ngrams, crie um modelo de 5-gramas e gere texto a partir dele.

Lembre-se que precisamos do seguinte para criar o modelo:

* Inserir tokens de inicio e fim de sentença adequados ao modelo, isto é, 4 tokens no início da sentença
* Criar listas de 4-gramas e 5-gramas usando o ngrams
* Contar os ngramas de cada lista
* Calcular a probabilidade adequadamente
* Criar o modelo em forma de dataframe
* Usar o modelo para gerar texto

Use o código acima como exemplo e crie o seu próprio modelo.

In [0]:
corpus = machado.sents('romance/marm05.txt')
corpus[:100]

[['Romance', ',', 'Memórias', 'Póstumas', 'de', 'Brás', 'Cubas', ',', '1880'], ['Memórias', 'Póstumas', 'de', 'Brás', 'Cubas'], ...]

In [0]:
words = []
for sent in corpus:
    sent.insert(0, '<s>')
    sent.insert(0, '<s>')
    sent.insert(0, '<s>')
    sent.insert(0, '<s>')
    sent.append('</s>')
    words.extend(sent)

In [0]:
n= 4
n2 = 5
forgram = ngrams(words, n)
fivegrams = ngrams(words, n2)

In [0]:
count_forgram = Counter(forgram)
count_forgram

Counter({('<s>', '<s>', '<s>', '<s>'): 3822,
         ('<s>', '<s>', '<s>', 'Romance'): 1,
         ('<s>', '<s>', 'Romance', ','): 1,
         ('<s>', 'Romance', ',', 'Memórias'): 1,
         ('Romance', ',', 'Memórias', 'Póstumas'): 1,
         (',', 'Memórias', 'Póstumas', 'de'): 1,
         ('Memórias', 'Póstumas', 'de', 'Brás'): 4,
         ('Póstumas', 'de', 'Brás', 'Cubas'): 4,
         ('de', 'Brás', 'Cubas', ','): 1,
         ('Brás', 'Cubas', ',', '1880'): 1,
         ('Cubas', ',', '1880', '</s>'): 1,
         (',', '1880', '</s>', '<s>'): 1,
         ('1880', '</s>', '<s>', '<s>'): 1,
         ('</s>', '<s>', '<s>', '<s>'): 3821,
         ('<s>', '<s>', '<s>', 'Memórias'): 1,
         ('<s>', '<s>', 'Memórias', 'Póstumas'): 1,
         ('<s>', 'Memórias', 'Póstumas', 'de'): 1,
         ('de', 'Brás', 'Cubas', '</s>'): 1,
         ('Brás', 'Cubas', '</s>', '<s>'): 1,
         ('Cubas', '</s>', '<s>', '<s>'): 1,
         ('<s>', '<s>', '<s>', 'Texto'): 1,
         ('<s>', '<s

In [0]:
count_fivegram = Counter(fivegrams)
count_fivegram

Counter({('<s>', '<s>', '<s>', '<s>', 'Romance'): 1,
         ('<s>', '<s>', '<s>', 'Romance', ','): 1,
         ('<s>', '<s>', 'Romance', ',', 'Memórias'): 1,
         ('<s>', 'Romance', ',', 'Memórias', 'Póstumas'): 1,
         ('Romance', ',', 'Memórias', 'Póstumas', 'de'): 1,
         (',', 'Memórias', 'Póstumas', 'de', 'Brás'): 1,
         ('Memórias', 'Póstumas', 'de', 'Brás', 'Cubas'): 4,
         ('Póstumas', 'de', 'Brás', 'Cubas', ','): 1,
         ('de', 'Brás', 'Cubas', ',', '1880'): 1,
         ('Brás', 'Cubas', ',', '1880', '</s>'): 1,
         ('Cubas', ',', '1880', '</s>', '<s>'): 1,
         (',', '1880', '</s>', '<s>', '<s>'): 1,
         ('1880', '</s>', '<s>', '<s>', '<s>'): 1,
         ('</s>', '<s>', '<s>', '<s>', '<s>'): 3821,
         ('<s>', '<s>', '<s>', '<s>', 'Memórias'): 1,
         ('<s>', '<s>', '<s>', 'Memórias', 'Póstumas'): 1,
         ('<s>', '<s>', 'Memórias', 'Póstumas', 'de'): 1,
         ('<s>', 'Memórias', 'Póstumas', 'de', 'Brás'): 1,
         ('

In [0]:
forgram_key = list(count_forgram.keys())
fivegram_key = list(count_fivegram.keys())
list_forgram = []
list_lastword = []
list_probs = []

    


In [0]:
list_forgram = []
list_lastword = []
list_probs = []

for t in fivegram_key:
    key_bigram = (t[0], t[1])
    prob = count_fivegram[t] / count_bigram[key_bigram]
    list_forgram.append(key_bigram)
    list_lastword.append(t[2])
    list_probs.append(prob)

In [0]:
model_df = pd.DataFrame({'ngram': list_forgram, 'lastword': list_lastword, 'prob': list_probs})
model_df

,ngram,lastword,prob
0,"(<s>, <s>)",<s>,0.000262
1,"(<s>, <s>)",<s>,0.000262
2,"(<s>, <s>)",Romance,0.000262
3,"(<s>, Romance)",",",1.000000
4,"(Romance, ,)",Memórias,1.000000
5,"(,, Memórias)",Póstumas,1.000000
6,"(Memórias, Póstumas)",de,0.800000
7,"(Póstumas, de)",Brás,0.250000
8,"(de, Brás)",Cubas,0.166667
9,"(Brás, Cubas)",",",0.034483


In [0]:
bigram_df = model_df.loc[model_df['ngram'] == ('Memórias', 'Póstumas')]
bigram_df

,ngram,lastword,prob
6,"(Memórias, Póstumas)",de,0.8
94,"(Memórias, Póstumas)",</s>,0.2


In [0]:
num_sents = 50
current_bigram = ('<s>', '<s>')
i = 0
while i < num_sents:
    df = model_df.loc[model_df['ngram'] == current_bigram]
    words = df['lastword'].values
    probs = df['prob'].values
    last_word = random.choices(words, probs)[0]
    
    current_bigram = (current_bigram[1], last_word)
    
    if last_word == '</s>':
        i+=1
    
    if last_word != '<s>' and last_word != '</s>':
        print(last_word, end=' ')

Perdoe - me o Dutra uma noite . Uma vida de D . Marcela como passou a noite , com o problema da vida particular uma cessação de movimento . O despropósito fez - me polidamente o refresco ; minha madrinha é a Excelentíssima Senhora D . Plácida acabaria como tantas outras criaturas humanas ; donde se poderia deduzir que o dito da rainha de Navarra , ocorre que , se não arrancarem de mim próprio ...  Menos um . Adeus ! Éramos dois rapazes , o pai ganhava apenas o necessário para endividar - se logo , à vista da notoriedade com que ele estremeceu e ficou . O que não hei de contá - lo e separamo - nos ? Unamos agora os pés , dão azo ao prazer de as impor aos outros , espiando pela rótula . Não esqueças que , de aflições que desabrochavam em alegria , segundo convinha a essa anônima , a qual é ainda a aliança do Damasceno , uma barafunda de coisas e cenas da meninice , buriladas na memória de quem o açucareiro ; e tudo escapava à compreensão do olhar humano , todos os motivos que levam o ho